In [2]:
import pandas as pd
import re
import numpy as np

In [3]:
unify_alleles = lambda x: re.sub('[*|:|-]', '', x)
intersect = lambda df1, df2: pd.merge(df1, df2, how='inner', on=["mhc", "sequence"], suffixes=["_curated", "_abelin"])

def merge(df1, df2):
    """
    pd.concat adds the two DataFrames together by appending one right after the other.
    if there is any overlap, it will be captured by the drop_duplicates method.
    However, drop_duplicates by default leaves the first observation and removes
    every other observation. In this case, we want every duplicate removed. 
    Hence, the keep=False parameter which does exactly that.
    
    A special note to the repeated df2. With only one df2 any row in df2 not in df1 won't
    be considered a duplicate and will remain. This solution with only one df2 only works
    when df2 is a subset of df1. However, if we concat df2 twice, it is guaranteed to be
    a duplicate and will subsequently be removed.
    """
    
    merged = pd.concat([df1, df2, df2], ignore_index=True).drop_duplicates(keep=False).reset_index(drop=True)
    print("{} samples in df1, {} samples in df2, {} samples in merged".format(df1.shape[0],
                                                                              df2.shape[0],
                                                                              merged.shape[0]))
    return merged



In [4]:
BIND_THR = 1 - np.log(500) / np.log(50000)
def log_meas(meas_values):
    meas_values = np.clip(meas_values, 1, 50000)
    return 1 - (np.log(meas_values) / np.log(50000))

binarize = lambda x: 1 if x >= BIND_THR else 0

### Curated

* Откроем *curated*, выбросим **явные** дубликаты, т.е. те семплы, которые совпадают с точностью до значащей цифры *meas*

In [16]:
curated = pd.read_csv("./curated.csv.gz")
curated = curated.drop_duplicates()

In [17]:
curated.shape, curated.drop_duplicates().shape, curated.drop_duplicates(keep=False).shape

((234943, 3), (234943, 3), (234943, 3))

* В полученном *curated* бинаризуем *meas* и запишем результат в *curated_bin*
* Полученный датафрейм содержит явные дупликаты для тех семплов, у которых был *близкий* meas

In [18]:
curated_bin = curated[["mhc", "sequence", "meas"]]
curated_bin.meas = log_meas(curated_bin.meas)
curated_bin.meas = curated_bin.meas.apply(binarize)

In [34]:
curated_bin.shape, curated_bin.drop_duplicates().shape, curated_bin.drop_duplicates(keep=False).shape

((234943, 3), (209115, 3), (185329, 3))

* Выберем из *curated* семплы, со **всеми** индексами дубликатов в curated_bin с помощью *curated[curated_bin.duplicated(keep=False)]}*
* Отсортируем полученную часть из *curated* по всем колонкам
* С помощью *drop_duplicates(["mhc", "sequence"], keep="first")* выбросим все дубликаты по колонкам *"mhc"* и *"sequence"*, оставив только первые встреченные - то есть семплы с минимальным *meas*, так как они были отсортированные ранее
* Тем самым, мы получили часть *curated*, в которой были близкие  по meas семплы, но относящиеся к одному классу
* Из этой части мы оставили семплы с минимальным *meas* и получили датафрейм *wo_inter*

In [69]:
wo_inter = curated[curated_bin.duplicated(keep=False)]
print("Shape of binarized duplicates: {}".format(wo_inter.shape))
wo_inter = wo_inter.sort_values(["mhc", "sequence", "meas"])
wo_inter = wo_inter.drop_duplicates(["mhc", "sequence"], keep="first")
print("Shape of curated part with duplicates of class, leaved with least meas: {}".format(wo_inter.shape))

Shape of binarized duplicates: (49614, 3)
Shape of curated part with duplicates of class, leaved with least meas: (23704, 3)


* Объеденим **полученную часть** с *curated*, из которого **полностью** выброшены дубликаты по классу 

In [47]:
new_curated = pd.concat([wo_inter, curated[~curated_bin.duplicated(keep=False)]])

* ####  Остались семплы, у которых после бинаризации разные классы - то есть предыдущий этап не схватил такие дубликаты. Выбросим их полностью, так как иначе трейн сет будет содержать одинаковые семплы с разными классами.

In [57]:
new_new_curated = new_curated.drop_duplicates(["mhc", "sequence"], keep=False)

In [71]:
new_new_curated.shape, \
new_new_curated.drop_duplicates().shape, \
new_new_curated.drop_duplicates(keep=False).shape

((199983, 3), (199983, 3), (199983, 3))

**Sanity check** : у полученного датафрейма *new_new_curated* бинаризовал, выбросил дубликаты, сравнил шейпы - получились ровно такие же как и у **исходного** *new_new_curated*

---
### ABELIN

* Откроем *abelin*, переименуем колонки
* ~~Бинаризуем *meas* у полученного ранее **чистого** *curated* (датафрейм *new_new_curated*) и найдем пересечение с ABELIN~~ **в этом нет необходимости, потому что находиться ровно столько же семплов и для небинаризованного**
---
* Пользуясь функцией *intersect* найдем пересечение по *"mhc"* и *"sequence"* в датафреймах *abelin* и *new_new_curated* и запишем это в датафрейм *cur_ab_inter*

In [75]:
abelin = pd.read_csv("abelin.csv.gz")
abelin = abelin[["hit", "allele", "peptide"]]
abelin.columns = ["meas", "mhc", "sequence"]

In [83]:
abelin.shape, abelin.drop_duplicates().shape, abelin.drop_duplicates(keep=False).shape

((2565753, 3), (2565753, 3), (2565753, 3))

In [93]:
cur_ab_inter = intersect(new_new_curated, abelin)
cur_ab_inter

,mhc,sequence,meas_curated,meas_abelin
0,HLA-A*01:01,ATDFKFAMY,0.740000,1
1,HLA-A*01:01,IADMGHLKY,3.200000,1
2,HLA-A*01:01,MIEPRTLQY,50.000000,1
3,HLA-A*01:01,STDHIPILY,1.700000,1
4,HLA-A*01:01,YLDDPDLKY,2.800000,1
5,HLA-A*01:01,YTAVVPLVY,0.300000,1
6,HLA-A*01:01,YTSDYFISY,5.300000,1
7,HLA-A*02:01,ALNELLQHV,50.000000,1
8,HLA-A*02:01,ALSNLEVKL,50.000000,1
9,HLA-A*02:01,ALWGFFPVL,2.700000,1


* Для поиска семплов с разными классами в исходных датафреймах, бинаризуем *meas_curated* в *cur_ab_inter* и запишем в *cur_ab_inter_bin* 
* В полученном *cur_ab_inter_bin* найдем семплы, для которых разные классы в исходных датафреймах и **выбросим из обоих датафреймов**
* Остальные семплы в пересечении (то есть для тех, у которых совпадают значения класса по *meas*)  **выбросим из *abelin***, то есть из теста, а в трейне оставим

In [95]:
cur_ab_inter_bin = cur_ab_inter[["mhc", "sequence", "meas_curated", "meas_abelin"]]
cur_ab_inter_bin.meas_curated = log_meas(cur_ab_inter_bin.meas_curated)
cur_ab_inter_bin.meas_curated = cur_ab_inter_bin.meas_curated.apply(binarize)

mislabels = cur_ab_inter[(cur_ab_inter_bin.meas_abelin != cur_ab_inter_bin.meas_curated)]
print("Number of mislabeled: {}".format(mislabels.shape))
mislabels.head()

Number of mislabeled: (14, 4)


,mhc,sequence,meas_curated,meas_abelin
25,HLA-A*02:03,GLRALRETL,23.0,0
32,HLA-A*02:01,ALLDKLYAL,1180.0,1
38,HLA-A*02:01,FLEQQNKIL,496.0,0
50,HLA-A*02:01,KMDASLGNLFA,1110.0,1
51,HLA-A*02:01,LLDRFLATV,560.0,1


In [97]:
from_abelin = cur_ab_inter[["mhc", "sequence", "meas_abelin"]]
from_abelin.columns = ["mhc", "sequence", "meas"]
abelin = merge(abelin, from_abelin)

2565753 samples in df1, 113 samples in df2, 2565640 samples in merged


In [99]:
from_curated = mislabels[["mhc", "sequence", "meas_curated"]]
from_curated.columns = ["mhc", "sequence", "meas"]
curated = merge(new_new_curated, from_curated)

199983 samples in df1, 14 samples in df2, 199969 samples in merged


### Sanity check
* проверим шейпы *curated* и *abelin* c на : *drop_duplicates(), drop_duplicates(keep=False)*
* проверим *intersect(abelin, curated)*
* Еще раз бинаризуем *meas* в *curated* и посмотрим на дубликаты, аналогично первому пункту

In [101]:
curated.shape, \
curated.drop_duplicates().shape, \
curated.drop_duplicates(keep=False).shape

((199969, 3), (199969, 3), (199969, 3))

In [102]:
abelin.shape, \
abelin.drop_duplicates().shape, \
abelin.drop_duplicates(keep=False).shape

((2565640, 3), (2565640, 3), (2565640, 3))

In [103]:
intersect(abelin, curated)

,meas_curated,mhc,sequence,meas_abelin


In [104]:
curated_bin_clean = curated[["mhc", "sequence", "meas"]]
curated_bin_clean.meas = log_meas(curated_bin_clean.meas)
curated_bin_clean.meas = curated_bin_clean.meas.apply(binarize)

In [105]:
curated_bin_clean.shape, \
curated_bin_clean.drop_duplicates().shape, \
curated_bin_clean.drop_duplicates(keep=False).shape

((199969, 3), (199969, 3), (199969, 3))

In [110]:
curated.to_csv("curated.csv.gz", compression="gzip", index=False)
abelin.to_csv("abelin.csv.gz", compression="gzip", index=False)

In [111]:
pd.read_csv("curated.csv.gz")

,mhc,sequence,meas
0,BoLA-6*13:01,VGYPKVKEEML,7.47
1,DLA-88*508:01,KLFSGELTK,50.00
2,DLA-88*508:01,RFLDKDGFIDK,50.00
3,ELA-A*01:01,RVEDVTNTAEYW,39.00
4,H-2-Db,AAHEFGHAL,100.00
5,H-2-Db,AALKNLCFY,2970.00
6,H-2-Db,AALRNLCFY,658.00
7,H-2-Db,AALRNLCFYS,98.80
8,H-2-Db,AALSKFKEDV,70000.00
9,H-2-Db,AAMLIRQGL,41700.00


------
### Curated + BData2009 - Blind

In [3]:
curated = pd.read_csv("./curated_training_data.csv")
print("samples in Curated: {}".format(curated.shape[0]))
curated = curated.drop_duplicates(keep=False, subset=["allele", "peptide", "measurement_value"]).reset_index(drop=True)
print("samples in Curated after dropping duplicates: {}".format(curated.shape[0]))

curated = curated[["allele", "peptide", "measurement_value"]]
curated.columns = ["mhc", "sequence", "meas"]
curated.mhc = curated.mhc.apply(unify_alleles)

curated["peptide_length"] = [len(i) for i in curated.sequence]
curated = curated[(curated.peptide_length >= 8) & (curated.peptide_length <= 15)]
print("samples in Curated after filter by length: {}".format(curated.shape[0]))
curated = curated[["mhc", "sequence", "meas"]]

samples in Curated: 241552
samples in Curated after dropping duplicates: 231103
samples in Curated after filter by length: 230078


In [4]:
bdata = pd.read_csv("bdata9.tsv", sep="\t")
bdata = bdata[["mhc", "sequence", "meas", "peptide_length"]]
bdata = bdata[(bdata.peptide_length >= 8) & (bdata.peptide_length <= 15)]
bdata = bdata[["mhc", "sequence", "meas"]]
bdata.mhc = bdata.mhc.apply(unify_alleles)
print("samples in Bdata2009: {}".format(bdata.shape[0]))

samples in Bdata2009: 137649


In [5]:
blind = pd.read_csv("blind.csv.gz")
blind.mhc = blind.mhc.apply(unify_alleles)
print("samples in blind: {}".format(blind.shape[0]))

samples in blind: 27680


In [6]:
inter_cur_bdata = intersect(curated, bdata)
print("Common samples in Curated and bdata: {}".format(inter_cur_bdata.shape[0]))
cur_wo_inter_wbdata = merge(curated, inter_cur_bdata)
print("Samples in curated without intersection with bdata: {}".format(cur_wo_inter_wbdata.shape[0]))
bdata_wo_inter_wcur = merge(bdata, inter_cur_bdata)
print("Samples in Bdata without intersection with curated: {}".format(bdata_wo_inter_wcur.shape[0]))
cur_bdata = pd.concat([cur_wo_inter_wbdata, bdata_wo_inter_wcur], ignore_index=True).reset_index(drop=True)
print("Samples in Curated + Bdata without intersection: {}".format(cur_bdata.shape[0]))

Common samples in Curated and bdata: 65284
230078 samples in df1, 65284 samples in df2, 164794 samples in merged
Samples in curated without intersection with bdata: 164794
137649 samples in df1, 65284 samples in df2, 72365 samples in merged
Samples in Bdata without intersection with curated: 72365
Samples in Curated + Bdata without intersection: 237159


In [7]:
inter_cur_bdata_blind = intersect(cur_bdata, blind)
print("Common samples in Curated + Bdata and Blind: {}".format(inter_cur_bdata_blind.shape[0]))
cur_bdata_wo_blind = merge(cur_bdata, blind)
print("Samples in (curated + bdata) without intersection with blind: {}".format(cur_bdata_wo_blind.shape[0]))

Common samples in Curated + Bdata and Blind: 17169
237159 samples in df1, 27680 samples in df2, 219990 samples in merged
Samples in (curated + bdata) without intersection with blind: 219990


In [8]:
cur_bdata_wo_blind[cur_bdata_wo_blind.duplicated()]

,mhc,sequence,meas


In [9]:
blind[blind.duplicated()]

,mhc,sequence,meas


In [10]:
intersect(cur_bdata_wo_blind, blind)

,mhc,sequence,meas


In [ ]:
cur_bdata_wo_blind.to_csv("./bdata2009_curated.csv.gz", compression="gzip", index=False)